# Downloading Files using HTTP Byte Range Requests with F#

## Calculating Byte Ranges using `Seq.unfold`

[`Seq.unfold`](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-collections-seqmodule.html#unfold) is used to generate a series from an initial value, stopping when a certain condition is met.

In our case:
1. the initial value is the desired byte range size
2. the stopping condition is when the next byte range start would be greater than the total file byte size

In [ ]:
let byteRanges (byteSize: uint64) (totalBytes: uint64) : (uint64 * uint64) seq =
    (0UL, byteSize)
    |> Seq.unfold (fun (start, finish) ->
        if start >= totalBytes then
            None
        else if start + byteSize > totalBytes then
            Some ((start, totalBytes-1UL),
                  (finish, finish + byteSize))
        else
            Some ((start, finish-1UL),
                  (finish, finish + byteSize))
    )

let tenMegabyteRanges = byteRanges 10_000_000UL

tenMegabyteRanges 15_000_000UL

index,Item1,Item2
0,0,9999999
1,10000000,14999999


The above example shows the byte size ranges calculated for downloading a 15 mb file in 10 mb chunks.

Let's step through each function call:

| # | `(start, finish)` | `start >= totalBytes` ? | `start + byteSize > totalBytes` ? | Outcome |
| - | ----------------- | ---------------------   | --------------------------------- | ------- |
| 0 | `(0UL, 10_000_000UL)` | `0UL >= 15_000_000UL = false` | `0UL + 10_000_000 > 15_000_000 = false` | `Some ((0UL, 9_999_999UL), (10_000_000UL, 20_000_000UL))` |